# Tutorial: Modelset loading

This notebook demonstrates how to use load and validate modelset configuration for sampling.

In [1]:
import yaml
from flumodelingsuite.sampling_validator import validate_sampling
from flumodelingsuite.calibration_validator import validate_calibration

***
## Sampling Modelset

### Load and validate YAML

In [6]:
# Load the YAML configuration file
with open('data/basic_modelset_sampling.yml', 'r') as file:
    sampling_config_txt = yaml.safe_load(file)

print(yaml.dump(sampling_config_txt))

modelset:
  meta:
    author: Flu Modeling Suite
    description: Vaccination scenarios for 2024
    version: 1.0.0
  population_names:
  - US-MA
  - US-NY
  sampling:
    compartments:
      I_symp:
        max: 0.1
        min: 0
      L:
        max: 0.25
        min: 0
    parameters:
      Rt:
        distribution:
          args:
          - 0.0
          - 2.0
          kwargs:
            some_other_param: 100
          name: uniform
          type: scipy
      gamma:
        values:
        - 0.1
        - 0.2
        - 0.3
        - 0.4
        - 0.5
      some_parameter_with_custom_dist:
        distribution:
          kwargs:
            arg1: 0.1
            arg2: 0.5
          name: my_custom_distribution
          type: custom
      vaccine_effectiveness:
        distribution:
          args:
          - 0.0
          - 0.5
          name: uniform
          type: scipy
    samplers:
    - parameters:
      - gamma
      strategy: grid
    - compartments:
      - L
      

In [ ]:
sampling_config = validate_sampling(sampling_config_txt)

In [18]:
print(f"Validated modelset name: {sampling_config.modelset.meta.description if sampling_config.modelset.meta else 'No description'}")
print(f"Number of populations: {len(sampling_config.modelset.population_names)}")
print(f"Number of samplers: {len(sampling_config.modelset.sampling.samplers)}")
print(f"Number of parameters: {len(sampling_config.modelset.sampling.parameters)}")

Validated modelset name: Vaccination scenarios for 2024
Number of populations: 2
Number of samplers: 2
Number of parameters: 4


### Values inside config

In [21]:
print("=== Samplers ===")
for i, sampler in enumerate(sampling_config.modelset.sampling.samplers):
    print(f"Sampler {i+1}:")
    print(f"  Strategy: {sampler.strategy}")
    print(f"  Parameters: {sampler.parameters}")
    if sampler.n_samples:
        print(f"  N Samples: {sampler.n_samples}")
    if sampler.compartments:
        print(f"  Compartments: {sampler.compartments}")
    print()

=== Samplers ===
Sampler 1:
  Strategy: grid
  Parameters: ['gamma']

Sampler 2:
  Strategy: LHS
  Parameters: ['start_date', 'Rt', 'vaccine_effectiveness']
  N Samples: 100
  Compartments: ['L', 'I_symp']



In [22]:
for param_name, param in sampling_config.modelset.sampling.parameters.items():
    print(f"{param_name}:")
    if param.distribution:
        print(f"  Distribution: {param.distribution.type} - {param.distribution.name}")
        print(f"  Args: {param.distribution.args}")
    if param.values:
        print(f"  Values: {param.values}")
    print()

Rt:
  Distribution: scipy - uniform
  Args: [0.0, 2.0]

vaccine_effectiveness:
  Distribution: scipy - uniform
  Args: [0.0, 0.5]

some_parameter_with_custom_dist:
  Distribution: custom - my_custom_distribution
  Args: []

gamma:
  Values: [0.1, 0.2, 0.3, 0.4, 0.5]



In [23]:
print("=== Compartments ===")
if sampling_config.modelset.sampling.compartments:
    for comp_name, comp_range in sampling_config.modelset.sampling.compartments.items():
        print(f"{comp_name}: min={comp_range.min}, max={comp_range.max}")
else:
    print("No compartment ranges defined")

=== Compartments ===
L: min=0.0, max=0.25
I_symp: min=0.0, max=0.1


In [24]:
print("=== Start Date ===")
if sampling_config.modelset.sampling.start_date:
    start_date = sampling_config.modelset.sampling.start_date
    print(f"Reference date: {start_date.reference_date}")
    print(f"Distribution: {start_date.distribution.type} - {start_date.distribution.name}")
    print(f"Args: {start_date.distribution.args}")
else:
    print("No start date configuration")

=== Start Date ===
Reference date: 2024-10-01
Distribution: scipy - randint
Args: [0.0, 30.0]


***
## Calibration modelset

### Load and validate YAML

In [10]:
# Load the YAML configuration file
with open('data/basic_modelset_calibration.yml', 'r') as file:
    calibration_config_txt = yaml.safe_load(file)

print(yaml.dump(calibration_config_txt))

modelset:
  calibration:
    comparison:
    - observed: hospitalized
      simulation:
      - Home_severe_to_Hospitalized_total
      - Home_severe_to_Hospitalized_vax_total
    distance_function: rmse
    observed_data_path: data/fludata.csv
    parameters:
      Rt:
        prior:
          args:
          - 1.0
          - 2.0
          kwargs:
            some_other_param: 100
          name: uniform
          type: scipy
      residual_immunity:
        prior:
          args:
          - 0.02
          - 0.35
          name: uniform
          type: scipy
      vaccine_effectiveness:
        prior:
          args:
          - 0.01
          - 0.5
          name: uniform
          type: scipy
    start_date:
      prior:
        args:
        - 0
        - 30
        name: randint
        type: scipy
      reference_date: 2024-10-01
    strategy:
      name: SMC
      options:
        num_generations: 5
        num_particles: 100
  meta:
    author: Flu Modeling Suite
    descript

In [11]:
calibration_config = validate_calibration(calibration_config_txt)

In [14]:
calibration_config

CalibrationConfig(modelset=CalibrationModelset(meta=Meta(description='Vaccination scenarios for 2024', author='Flu Modeling Suite', version='1.0.0'), population_names=['US-MA', 'US-NY'], calibration=CalibrationConfiguration(strategy=CalibrationStrategy(name='SMC', options={'num_particles': 100, 'num_generations': 5}), distance_function='rmse', observed_data_path='data/fludata.csv', comparison=[ComparisonSpec(observed='hospitalized', simulation=['Home_severe_to_Hospitalized_total', 'Home_severe_to_Hospitalized_vax_total'])], start_date=DateParameter(reference_date=datetime.date(2024, 10, 1), distribution=None, prior=Distribution(type='scipy', name='randint', args=[0.0, 30.0], kwargs={})), parameters={'Rt': CalibrationParameter(prior=Distribution(type='scipy', name='uniform', args=[1.0, 2.0], kwargs={'some_other_param': 100})), 'residual_immunity': CalibrationParameter(prior=Distribution(type='scipy', name='uniform', args=[0.02, 0.35], kwargs={})), 'vaccine_effectiveness': CalibrationPar

In [22]:
print("=== Calibration ===")

print(f"Strategy:")
print(f"  Name: {calibration_config.modelset.calibration.strategy.name}")
print(f"  Options: {calibration_config.modelset.calibration.strategy.options}")


print(f"Distance Function: {calibration_config.modelset.calibration.distance_function}")
print(f"Observed Data Path: {calibration_config.modelset.calibration.observed_data_path}")

print(f"\nComparison:")
for i, comp in enumerate(calibration_config.modelset.calibration.comparison):
    print(f"  {i+1}. Observed: {comp.observed}")
    print(f"     Simulation: {comp.simulation}")

print(f"\nStart Date:")
if calibration_config.modelset.calibration.start_date:
    start_date = calibration_config.modelset.calibration.start_date
    print(f"  Reference Date: {start_date.reference_date}")
    if start_date.prior:
        print(f"  Prior: {start_date.prior.type} - {start_date.prior.name}")
        print(f"  Args: {start_date.prior.args}")

print(f"\nParameters:")
for param_name, param in calibration_config.modelset.calibration.parameters.items():
    print(f"  {param_name}:")
    if param.prior:
        print(f"    Prior: {param.prior.type} - {param.prior.name}")
        print(f"    Args: {param.prior.args}")
        if param.prior.kwargs:
            print(f"    Kwargs: {param.prior.kwargs}")

=== Calibration ===
Strategy:
  Name: SMC
  Options: {'num_particles': 100, 'num_generations': 5}
Distance Function: rmse
Observed Data Path: data/fludata.csv

Comparison:
  1. Observed: hospitalized
     Simulation: ['Home_severe_to_Hospitalized_total', 'Home_severe_to_Hospitalized_vax_total']

Start Date:
  Reference Date: 2024-10-01
  Prior: scipy - randint
  Args: [0.0, 30.0]

Parameters:
  Rt:
    Prior: scipy - uniform
    Args: [1.0, 2.0]
    Kwargs: {'some_other_param': 100}
  residual_immunity:
    Prior: scipy - uniform
    Args: [0.02, 0.35]
  vaccine_effectiveness:
    Prior: scipy - uniform
    Args: [0.01, 0.5]
